## Question 1: dlt Version

```bash
@dexhresthai ➜ /workspaces/data-engineering-practice (main) $ dlt --version
dlt 1.6.1
```


# Question 2: Define & Run the Pipeline (NYC Taxi API)
Use dlt to extract all pages of data from the API.

Steps:

1️⃣ Use the @dlt.resource decorator to define the API source.

2️⃣ Implement automatic pagination using dlt's built-in REST client.

3️⃣ Load the extracted data into DuckDB for querying.

How many tables were created?  
Ans : 4

In [1]:

import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator


# your code is here
BASE_URL = "https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api"

@dlt.resource(name="rides")
def ny_taxi():
    client = RESTClient(
        base_url=BASE_URL,
        paginator=PageNumberPaginator(
            base_page=1,
            total_path=None
        )
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):    # <--- API endpoint for retrieving taxi ride data
        yield page 

pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline",
    destination="duckdb",
    dataset_name="ny_taxi_data"
)


load_info = pipeline.run(ny_taxi,  write_disposition="replace")
print(load_info)

2025-02-15 23:24:09,147|[WARNING]|23378|127460152345216|dlt|configuration.py|_path_from_pipeline:178|Duckdb attached to pipeline ny_taxi_pipeline in path ny_taxi_pipeline.duckdb was could not be found but pipeline has already ran. This may be a result of (1) recreating or attaching pipeline  without or with changed explicit path to database that was used when creating the pipeline. (2) keeping the path to to database in secrets and changing the current working folder so  dlt cannot see them. (3) you deleting the database.


Pipeline ny_taxi_pipeline load step completed in 1.23 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:////workspaces/data-engineering-practice/Week 4/notebooks/ny_taxi_pipeline.duckdb location to store data
Load package 1739661849.2181392 is LOADED and contains no failed jobs


In [2]:
import duckdb
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,ny_taxi_pipeline,ny_taxi_data,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,ny_taxi_pipeline,ny_taxi_data,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,ny_taxi_pipeline,ny_taxi_data,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,ny_taxi_pipeline,ny_taxi_data,rides,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


# Question 3 : Explore the loaded data

What is the total number of records extracted?   
Ans : 10000

In [3]:
display(pipeline.dataset(dataset_type="default").rides.df().head())
print("Total records: ",pipeline.dataset(dataset_type="default").rides.df().shape[0])

,end_lat,end_lon,fare_amt,passenger_count,payment_type,start_lat,start_lon,tip_amt,tolls_amt,total_amt,trip_distance,trip_dropoff_date_time,trip_pickup_date_time,surcharge,vendor_name,_dlt_load_id,_dlt_id,store_and_forward
0,40.742963,-73.980072,45.0,1,Credit,40.641525,-73.787442,9.0,4.15,58.15,17.52,2009-06-14 23:48:00+00:00,2009-06-14 23:23:00+00:00,0.0,VTS,1739661849.2181392,TzSHgu8zgUhW7g,NaN
1,40.740187,-74.005698,6.5,1,Credit,40.722065,-74.009767,1.0,0.00,8.50,1.56,2009-06-18 17:43:00+00:00,2009-06-18 17:35:00+00:00,1.0,VTS,1739661849.2181392,CMCY+gXOLx3/ug,NaN
2,40.718043,-74.004745,12.5,5,Credit,40.761945,-73.983038,2.0,0.00,15.50,3.37,2009-06-10 18:27:00+00:00,2009-06-10 18:08:00+00:00,1.0,VTS,1739661849.2181392,nWu+8ButULYEJw,NaN
3,40.739637,-73.985233,4.9,1,CASH,40.749802,-73.992247,0.0,0.00,5.40,1.11,2009-06-14 23:58:00+00:00,2009-06-14 23:54:00+00:00,0.5,VTS,1739661849.2181392,UDw8yW2lGSPbgg,NaN
4,40.730032,-73.852693,25.7,1,CASH,40.776825,-73.949233,0.0,4.15,29.85,11.09,2009-06-13 13:23:00+00:00,2009-06-13 13:01:00+00:00,0.0,VTS,1739661849.2181392,qYQ3XMZ8LUGYJw,NaN


Total records:  10000


# Question 4 : Trip Duration Analysis

What is the average trip duration?  
Ans : 12.3049


In [4]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """
        )
    # Prints column values of the first row
    print(res)

[(12.3049,)]
